In [1]:
res_list = []

In [2]:
batch_size = 256*4
folder_name = 'demo//'
import mxnet as mx
import cv2
import numpy as np

In [3]:
def do_multi_predict(model_name,epoch_num,tt):
    print(model_name,epoch_num)
    batch_size = 256*4
    #sym, arg_params, aux_params = mx.model.load_checkpoint('fine-tuned-firstclass-res18-use-resize', 25)
    sym, arg_params, aux_params = mx.model.load_checkpoint(model_name,epoch_num)
    mod = mx.mod.Module(symbol=sym, context=mx.gpu(), label_names=None)
    longth_ = int(160/8*7)
    width_ = int(110/8*7)
    mod.bind(for_training=False, data_shapes=[('data', (batch_size,3,longth_,width_))], 
             label_shapes=mod._label_shapes)
    mod.set_params(arg_params, aux_params, allow_missing=True)
    augs = mx.image.CreateAugmenter(data_shape=(3,longth_, width_),rand_crop=True,rand_resize=False, rand_mirror=True, brightness=0.125, contrast=0.125, rand_gray=0.05,saturation=0.125, pca_noise=0, inter_method=10)
    final_res = None
    # define a simple data batch
    for i in range(tt):
        print(i)
        val_iter= mx.image.ImageIter(batch_size=batch_size, data_shape=(3, longth_, width_), label_width=1,
                                       path_imgidx=folder_name+'cloth_val.idx', path_imgrec=folder_name+'cloth_val.rec', shuffle=False,
                                       aug_list=augs)
        res = mod.predict(val_iter,always_output_list=True,num_batch=10)
        if final_res is None:
            final_res = res
        else:
            final_res += res
    del mod
    return final_res
#res_list.append(do_multi_predict('fine-tuned-firstclass-res50-use-resize-use-lr_sch', 65,20))
#res_list.append(do_multi_predict('fine-tuned-firstclass-res18-use-resize', 25,20))

In [4]:
res_list.append(do_multi_predict('fine-tuned-firstclass-res152-use-resize-use-lr_sch', 30,10))


fine-tuned-firstclass-res152-use-resize-use-lr_sch 30
0
1
2
3
4
5
6
7
8
9


In [3]:
import pickle
with open('./model_avage_grab.pickle','rb') as f:
    res_list = pickle.load(f)
print(len(res_list[0]))
'''
with open('model_avage_grab.pickle','wb') as f:
    pickle.dump(res_list,f)
'''

10


"\nwith open('model_avage_grab.pickle','wb') as f:\n    pickle.dump(res_list,f)\n"

In [2]:
batch_size = 256 * 8
#sym, arg_params, aux_params = mx.model.load_checkpoint('fine-tuned-firstclass-res18-use-resize', 25)
sym, arg_params, aux_params = mx.model.load_checkpoint('fine-tuned-firstclass-res50-use-resize-use-lr_sch', 65)
mod = mx.mod.Module(symbol=sym, context=mx.gpu(), label_names=None)
longth_ = int(160/8*7)
width_ = int(110/8*7)
mod.bind(for_training=False, data_shapes=[('data', (batch_size,3,longth_,width_))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)


## Predicting

We first define helper functions for downloading an image and performing the
prediction:

In [3]:
folder_name = 'first_class_data_nopre//'
augs = mx.image.CreateAugmenter(data_shape=(3,longth_, width_),rand_crop=True,rand_resize=False, rand_mirror=True, brightness=0.125, contrast=0.125, rand_gray=0.05,saturation=0.125, pca_noise=0, inter_method=10)
final_res = None

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])
tt = 10
for i in range(tt):
    val_iter= mx.image.ImageIter(batch_size=batch_size, data_shape=(3, longth_, width_), label_width=1,
                                   path_imgidx=folder_name+'cloth_val.idx', path_imgrec=folder_name+'cloth_val.rec', shuffle=False,
                                   aug_list=augs)
    res = mod.predict(val_iter,always_output_list=True,num_batch=10)
    if final_res is None:
        final_res = res
    else:
        final_res += res
'''
def predict(url):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], labels[i]))
'''

"\ndef predict(url):\n    img = get_image(url, show=True)\n    # compute the predict probabilities\n    mod.forward(Batch([mx.nd.array(img)]))\n    prob = mod.get_outputs()[0].asnumpy()\n    # print the top-5\n    prob = np.squeeze(prob)\n    a = np.argsort(prob)[::-1]\n    for i in a[0:5]:\n        print('probability=%f, class=%s' %(prob[i], labels[i]))\n"

In [6]:
longth_ = int(160/8*7)
width_ = int(110/8*7)
augs = mx.image.CreateAugmenter(data_shape=(3,longth_, width_),rand_crop=True,rand_resize=False, rand_mirror=True, brightness=0.125, contrast=0.125, rand_gray=0.05,saturation=0.125, pca_noise=0, inter_method=10)
val_iter= mx.image.ImageIter(batch_size=batch_size, data_shape=(3, longth_, width_), label_width=1,
                               path_imgidx=folder_name+'cloth_val.idx', path_imgrec=folder_name+'cloth_val.rec', shuffle=False,
                               aug_list=augs)
val_label = None
for batch in val_iter:
    l = batch.label[0]
    if val_label is None:
        val_label = l
    else:
        val_label = mx.ndarray.concat(val_label,l,dim=0)
    #print(mx.ndarray.concat(l,l,dim=1))
val_label = val_label[:7108].as_in_context(mx.gpu(0))
print(val_label)


[ 7.  1.  0. ...,  4.  4.  1.]
<NDArray 7108 @gpu(0)>


In [ ]:
print(res_list[0][0].shape)

(7108, 8)


In [8]:
def eval_avage(fea_list):
    res_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.gpu())
    #res_final = final_res[0]
    for i in fea_list:
        res_final += i
    res_final/= len(fea_list)
    acc = mx.metric.Accuracy()
    top2 = mx.metric.TopKAccuracy(top_k=2)
    acc.update(labels=[val_label],preds=[res_final])
    top2.update(labels=[val_label],preds=[res_final])
    print(acc.get())
    print(top2.get())
def eval_avage_thre(fea_list):
    res_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.gpu())
    #res_final = final_res[0]
    for i in fea_list:
        res_final += i
    res_final/= len(fea_list)
    acc = mx.metric.Accuracy()
    top2 = mx.metric.TopKAccuracy(top_k=2)
    acc.update(labels=[val_label],preds=[res_final])
    top2.update(labels=[val_label],preds=[res_final])
    print(acc.get())
    print(top2.get())
eval_avage_thre(  res_list[0])

('accuracy', 0.63956105796285878)
('top_k_accuracy_2', 0.78474957794034894)


In [65]:
fea_list = res_list[2]
res_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.gpu())
#res_final = final_res[0]
for i in fea_list:
    res_final += i
res_final/= len(fea_list)
dd= res_final[3]
acc = mx.metric.Accuracy()
top2 = mx.metric.TopKAccuracy(top_k=2)
top3 = mx.metric.TopKAccuracy(top_k=3)
acc.update(labels=[val_label],preds=[res_final])
top2.update(labels=[val_label],preds=[res_final])
top3.update(labels=[val_label],preds=[res_final])
print(acc.get())
print(top2.get())
print(top3.get())

('accuracy', 0.93415869442881261)
('top_k_accuracy_2', 0.9800225098480585)
('top_k_accuracy_3', 0.99071468767585824)


In [66]:
p = res_final.asnumpy()
l = val_label.asnumpy()

In [129]:
ff = []
tt = []
fff = []
ttf = []
allf = []
for i in range(len(p)):
    pp = p[i]
    mm = pp.argmax()
    second = (pp.argsort()[-3:])
    if mm != l[i]:
        allf.append(pp[mm])
    if mm != l[i] and pp[mm] <0.75:
        ff.append(pp[mm])
        if l[i] not in second:
            fff.append(pp[mm])
    else:
        if pp[mm] < 0.75:
            tt.append(pp[mm])
            if l[i] not in second:
                ttf.append(pp[mm])
print(len(tt),len(ff),len(ttf),len(fff),len(allf))
print((len(tt)+len(ff))/7108)
#print(tt)
print((len(allf)+len(fff)-len(ff))/7108)

399 285 0 34 468
0.09622960045019696
0.03052898142937535


In [30]:
acc = mx.metric.Accuracy()
top2 = mx.metric.TopKAccuracy(top_k=2)
acc.update(labels=[val_label[:7108]],preds=[res_final])
top2.update(labels=[val_label[:7108]],preds=[res_final])
print(acc.get())
print(top2.get())

('accuracy', 0.88716938660664035)
('top_k_accuracy_2', 0.95779403489026449)


In [16]:
res1 = res[0] + res[0]
zero = np.zeros((7108,8))
zero.shape

(7108, 8)

In [6]:
def do_multi_predict(model_name,epoch_num,tt):
    print(model_name,epoch_num)
    batch_size = 1
    #sym, arg_params, aux_params = mx.model.load_checkpoint('fine-tuned-firstclass-res18-use-resize', 25)
    sym, arg_params, aux_params = mx.model.load_checkpoint(model_name,epoch_num)
    mod = mx.mod.Module(symbol=sym, context=mx.cpu(), label_names=None)
    longth_ = int(160/8*7)
    width_ = int(110/8*7)
    mod.bind(for_training=False, data_shapes=[('data', (batch_size,3,longth_,width_))], 
             label_shapes=mod._label_shapes)
    mod.set_params(arg_params, aux_params, allow_missing=True)
    augs = mx.image.CreateAugmenter(data_shape=(3,longth_, width_),rand_crop=True,rand_resize=False, rand_mirror=True, brightness=0.125, contrast=0.125, rand_gray=0.05,saturation=0.125, pca_noise=0, inter_method=10)
    final_res = None
    # define a simple data batch
    for i in range(tt):
        val_iter= mx.image.ImageIter(batch_size=batch_size, data_shape=(3, longth_, width_), label_width=1,
                                       path_imgidx=folder_name+'cloth_val.idx', path_imgrec=folder_name+'cloth_val.rec', shuffle=False,
                                       aug_list=augs)
        res = mod.predict(val_iter,always_output_list=True)
        if final_res is None:
            final_res = res
        else:
            final_res += res
    del mod
    return final_res
%time fea_list = (do_multi_predict('fine-tuned-firstclass-res152-use-resize-use-lr_sch', 30,20))

res_final = mx.nd.zeros(fea_list[0].shape,ctx=mx.cpu())
#res_final = final_res[0]
for i in fea_list:
    res_final += i
res_final/= len(fea_list)
print(1,10,3,4,5,6,7,8)
rrrr =  ['帽子','鞋子','披带类','上装','裤子','裙子','连体装','包包']
print(res_final,rrrr[res_final[0].asnumpy().argmax()])

fine-tuned-firstclass-res152-use-resize-use-lr_sch 30
CPU times: user 280 ms, sys: 68 ms, total: 348 ms
Wall time: 241 ms
1 10 3 4 5 6 7 8

[[  8.17690961e-05   4.03449150e-08   4.80921926e-07   9.99914527e-01
    2.71953180e-08   1.71702993e-06   1.31430693e-06   5.71130485e-08]]
<NDArray 1x8 @cpu(0)> 上装



<!-- INSERT SOURCE DOWNLOAD BUTTONS -->

